# sk8s: Simple Kubernetes in python

Here is a demo of the `sk8s` library, which exposes the primary functions of kubernetes as a python API.

- Run python lambdas as k8s Jobs.
- Implement workflows comprised of interdependent Jobs as simple python functions.
- Create and destroy k8s Volumes, directly or via Jobs and workflows.
- Run python lambdas as k8s Services, e.g. Flask apps.

# Prerequesites
- Access to a kubernetes cluster. This demo is routinely run on both GKE, and Docker Desktop's local k8s cluster. If `kubectl get all` works, you're probably all set.
- `sk8s` module installed in your environment. `pip install -e .` in the root of this repo is how I install it. 

# sk8s Demo

In [13]:
import sk8s

### Run a job and collect the result

In [14]:
result = sk8s.wait(sk8s.run(lambda: "Hooray"), timeout=500)
assert(result=="Hooray")
print(f"Result should be 'Hooray', result is '{result}'")

Result should be 'Hooray', result is 'Hooray'


### Run many jobs via map and collect the results

In [15]:
results = sk8s.map(lambda i: i*2, (0,1,2))
assert(tuple(results) == (0,2,4))
print(f"results should be [0,2,4], results are {results}")

results should be [0,2,4], results are [0, 2, 4]


### Execute a simple workflow

In [16]:
def wf():
    jobs1 = sk8s.map(lambda i: i, range(3), asynchro=True)
    return sk8s.wait(sk8s.run(lambda inputs: sum(inputs),
                              map(sk8s.wait, jobs1),
                              name="job2-{s}",
                              asynchro=True),
                     timeout=500)
result = sk8s.wait(sk8s.run(wf, name="wf-{s}"), timeout=500)
assert(result == 3)
print(f"result should be 3, result is {result}")

result should be 3, result is 3


### Create a volume, and use it in a workflow

In [17]:
def wf():
    import json
    volume = sk8s.create_volume("10Mi", accessModes=["ReadWriteOnce"])

    def func1():
        with open(f"/mnt/{volume}/test.json", "w") as fp:
            json.dump("hey", fp)

    def func2():
        with open(f"/mnt/{volume}/test.json") as fp:
            return json.load(fp)

    sk8s.wait(sk8s.run(func1, volumes=[volume]))
    result = sk8s.wait(sk8s.run(func2, volumes=[volume]))

    sk8s.delete_volume(volume)
                              
    return result

result = wf()
assert(result == "hey")
print(f'result should be "hey", result is "{result}"')

result should be "hey", result is "hey"
